## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import random
from deap import base
from deap import creator
from deap import tools

## Setting Goal Energy / Goal Protein

In [2]:
days = int(input("Day: "))
weight = float(input("Weight: "))
energy_coeff = float(input("Energy Coefficient: ")) 
protein_coeff = float(input("Protein Coefficient: ")) 
total_calories = energy_coeff*weight*days

gram_prot = protein_coeff*weight*days

percentage_prot = (gram_prot*4)/total_calories
percentage_carb= 0.5
percentage_fat = 1-(percentage_prot+percentage_carb)

cal_prot = round(gram_prot*4)
cal_carb = round(percentage_carb * total_calories)
cal_fat = round(percentage_fat * total_calories)

carb_cal_p_gram = 4
fat_cal_p_gram = 9

gram_carb = cal_carb / carb_cal_p_gram
gram_fat = cal_fat / fat_cal_p_gram

print(total_calories, "\n",
     gram_prot, gram_carb, gram_fat, "\n",
     percentage_prot, percentage_carb, percentage_fat, "\n",
     cal_prot, cal_carb, cal_fat)

Day: 1
Weight: 68
Energy Coefficient: 35
Protein Coefficient: 1.5
2380.0 
 102.0 297.5 86.88888888888889 
 0.17142857142857143 0.5 0.3285714285714285 
 408 1190 782


## Import Data

In [3]:
products_table = pd.read_csv("data/foodexchange.csv")

In [4]:
products_table

,food group,carbohydrate,protien,lipid,energy
0,นมไขมันเต็มส่วน,12,8,8,150
1,นมพร่องมันเนย,12,8,5,120
2,นมขาดมันเนย,12,8,0,80
3,ผักให้พลังงานน้อย,0,0,0,0
4,ผักให้พลังงาน,5,2,0,25
5,ผลไม้,15,0,0,60
6,ข้าวแป้ง,18,2,0,80
7,เนื้อสัตว์ไขมันต่ำมาก,0,7,1,35
8,เนื้อสัตว์ไขมันต่ำ,0,7,3,55
9,เนื้อสัตว์ไขมันปานกลาง,0,7,5,75


## Setting Food intake

In [ ]:
food_intake = []
for i in products_table["food group"]:
    x = int(input(i + ": "))
    food_intake.append(x)

total_calories = total_calories -sum(x*y for x,y in zip(food_intake, products_table["energy"]))
gram_carb = gram_carb - sum(x*y for x,y in zip(food_intake, products_table["carbohydrate"]))
gram_prot = gram_prot - sum(x*y for x,y in zip(food_intake, products_table["protien"]))
gram_fat = gram_fat - sum(x*y for x,y in zip(food_intake, products_table["lipid"]))


cal_prot = round(gram_prot*4)
cal_carb = round(gram_carb * 4)
cal_fat = round(gram_fat * 9)

## Optimize The Shopping List Multivariately To Match Calories, Protein

In [5]:
# extract the information of products in a format that is easier to use in the deap algorithms cost function
cal_data = list(products_table['energy'])
prot_data = list(products_table['protien'])
fat_data = list(products_table['lipid'])
carb_data = list(products_table['carbohydrate'])

In [6]:
def n_per_product():
    return random.choices( range(0, 20), k = 12)

In [7]:
# in this second version, we optimize for the 2 components of the shopping list: calories, protein
# if we need to make everything as important, we should add a weight to them
# we know that there are 30% protein calories, 20% fat and 50% carbs.
weights = (-1,-1/percentage_prot,-1/percentage_fat,-1/percentage_carb)
weights2 = (1,1,1,1)
weights3 = (-1., -1.)
weights4 = (-1,-1,-1,-1)

In [8]:
creator.create("FitnessMin", base.Fitness, weights=weights3)
creator.create("Individual", list, fitness=creator.FitnessMin)

In [9]:
def evaluate(individual):
    individual = individual[0]
    tot_prot = sum(x*y for x,y in zip(prot_data,individual)) *4
    tot_fat = sum(x*y for x,y in zip(fat_data,individual)) *9
    tot_carb = sum(x*y for x,y in zip(carb_data,individual)) *4 
    cals = tot_prot + tot_carb + tot_fat
    
    
    return  abs(cals - total_calories)  , \
           abs(tot_prot - cal_prot ) , \
            

In [10]:
# this is the setup of the deap library: registering the different function into the toolbox
toolbox = base.Toolbox()

toolbox.register("n_per_product", n_per_product)

toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.n_per_product, n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, indpb=0.05, low = 0, up = 10)
toolbox.register("select", tools.selTournament, tournsize=3)


In [11]:
def main():
    pop = toolbox.population(n=300)

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    CXPB, MUTPB = 0.7, 0.3
    
    # Extracting all the fitnesses of 
    fits = [ind.fitness.values[0] for ind in pop]
    
    # Variable keeping track of the number of generations
    g = 0
    
    fit_score = min([sum([toolbox.evaluate(x)[0]**2 /scales_cals**2, toolbox.evaluate(x)[1]**2 /scales_prot**2 ]) for x in pop])
    
    # Begin the evolution
    while fit_score > 0.0004:
        # A new generation
        g = g + 1
        #print("-- Generation %i --" % g)
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
        
                # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1[0], child2[0])
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant[0])
                del mutant.fitness.values
            
                
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
            
        pop[:] = offspring
        # print(offspring)
        fit_score = min([sum([toolbox.evaluate(x)[0]**2 /scales_cals**2, toolbox.evaluate(x)[1]**2 /scales_prot**2 ]) for x in pop])
        min_ind = pop[np.argmin([sum([toolbox.evaluate(x)[0]**2 /scales_cals**2, toolbox.evaluate(x)[1]**2 /scales_prot**2 ]) for x in pop])]
        print(min_ind)
        #print(fit_score)
        
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        #print(min(fits), max(fits), mean, std)
    #print(offspring)
    
    best = pop[np.argmin([sum([toolbox.evaluate(x)[0]**2 /scales_cals**2, toolbox.evaluate(x)[1]**2 /scales_prot**2 ]) for x in pop])]
    return best

In [12]:
best_solution = main()

[[1, 1, 2, 13, 6, 9, 2, 13, 0, 2, 7, 7]]
[[1, 0, 6, 3, 1, 10, 5, 1, 0, 0, 5, 3]]


In [68]:
products_table['multivariate_choice'] = pd.Series(best_solution[0])
products_table

,food group,carbohydrate,protien,lipid,energy,multivariate_choice,multivariate_gr_prot,multivariate_gr_fat,multivariate_gr_carb,multivariate_cal
0,นมไขมันเต็มส่วน,12,8,8,150,2,72,72,108,1350
1,นมพร่องมันเนย,12,8,5,120,4,0,0,0,0
2,นมขาดมันเนย,12,8,0,80,0,0,0,0,0
3,ผักให้พลังงานน้อย,0,0,0,0,18,0,0,0,0
4,ผักให้พลังงาน,5,2,0,25,2,2,0,5,25
5,ผลไม้,15,0,0,60,6,0,0,135,540
6,ข้าวแป้ง,18,2,0,80,5,0,0,0,0
7,เนื้อสัตว์ไขมันต่ำมาก,0,7,1,35,0,14,2,0,70
8,เนื้อสัตว์ไขมันต่ำ,0,7,3,55,4,7,3,0,55
9,เนื้อสัตว์ไขมันปานกลาง,0,7,5,75,0,7,5,0,75


In [21]:
    pop = toolbox.population(n=9)
    prot_list = []
    carb_list = []
    fat_list = []
    cal_list = []
    for ind in pop:
        prot = sum(x*y for x,y in zip(prot_data,ind[0]))*4
        carb = sum(x*y for x,y in zip(carb_data,ind[0])) *4 
        fat = sum(x*y for x,y in zip(fat_data,ind[0])) *9
        cals = sum(x*y for x,y in zip(cal_data,ind[0]))
        carb_list.append(carb)
        fat_list.append(fat)
        prot_list.append(prot)
        cal_list.append(cals)

    scales_carb = max(carb_list)-min(carb_list)
    scales_fat = max(fat_list)-min(fat_list)
    scales_prot = max(prot_list)-min(prot_list)
    scales_cals = max(cal_list)-min(cal_list)
print(pop)

[[[10, 19, 6, 14, 6, 15, 10, 19, 11, 18, 0, 13]], [[1, 0, 14, 13, 11, 2, 13, 17, 0, 1, 13, 14]], [[15, 8, 13, 1, 8, 17, 13, 4, 9, 3, 13, 2]], [[14, 2, 10, 9, 4, 7, 12, 16, 14, 5, 17, 18]], [[13, 14, 7, 1, 10, 7, 7, 15, 18, 19, 7, 14]], [[8, 2, 9, 17, 17, 6, 13, 6, 8, 0, 9, 13]], [[10, 4, 7, 2, 16, 18, 16, 6, 4, 0, 1, 9]], [[11, 16, 5, 16, 18, 11, 13, 7, 14, 1, 11, 14]], [[8, 15, 12, 4, 19, 15, 8, 14, 9, 14, 14, 19]]]


In [22]:
([sum([toolbox.evaluate(x)[0]**2 /scales_cals**2, toolbox.evaluate(x)[1]**2 /scales_prot**2 ]) for x in pop]) 

[3.8052187344856607,
 0.8403180184234522,
 2.8097717500205825,
 3.23079644237336,
 4.507405730463075,
 1.0328354024390631,
 0.8907151652252001,
 3.0285927449532357,
 4.686740613345869]

## Analyzing

In [65]:
products_table['multivariate_gr_prot'] = products_table['multivariate_choice'] * products_table['protien']
products_table['multivariate_gr_fat'] = products_table['multivariate_choice'] * products_table['lipid']
products_table['multivariate_gr_carb'] = products_table['multivariate_choice'] * products_table['carbohydrate']
products_table['multivariate_cal'] = products_table['multivariate_choice'] * products_table['energy']

In [66]:
summary = pd.DataFrame.from_records(
[
    [products_table['multivariate_gr_prot'].sum(), gram_prot],

    [products_table['multivariate_cal'].sum(), total_calories]
])
summary.columns = ['multivariate', 'goal']
summary.index = ['prot',  'cal']
summary["multiv_error"] = (summary["goal"] - summary["multivariate"]).apply(abs) / summary["goal"]
summary

,multivariate,goal,multiv_error
prot,109,105.0,0.038095
cal,2440,2450.0,0.004082
